In [2]:
import requests
import pandas as pd
import matplotlib as plt
import numpy as np


In [34]:
#open bus data api key
API_KEY = 'enter OpenBusData api key'

base_url = 'https://data.bus-data.dft.gov.uk/api/v1/datafeed/'


We have around 750-800 buses between AMSY and SCMY

## starts here

Below saves the stops data for each route in a file.

In [15]:
import os
import json
import xml.etree.ElementTree as ET

# extracts stops from an XML file
def extract_stops_from_xml(xml_file):
    namespace = {'txc': 'http://www.transxchange.org.uk/'}
    tree = ET.parse(xml_file)
    root = tree.getroot()

    stops = {}

    # loops through stop points and gathers info
    for stop_point in root.findall('.//txc:StopPoints/txc:AnnotatedStopPointRef', namespaces=namespace):
        ref = stop_point.find('txc:StopPointRef', namespaces=namespace).text
        common_name = stop_point.find('txc:CommonName', namespaces=namespace).text
        lon = float(stop_point.find('.//txc:Location/txc:Longitude', namespaces=namespace).text)
        lat = float(stop_point.find('.//txc:Location/txc:Latitude', namespaces=namespace).text)

        stops[ref] = {'common_name': common_name, 'coordinates': (lon, lat)}

    return stops

# processes a single file and saves stops to a JSON file
def process_single_file(file_path, output_directory):
    route_id = get_route_id_from_filename(file_path)
    if route_id:
        stops = extract_stops_from_xml(file_path)
        output_file = os.path.join(output_directory, f'{route_id}_stops.json')
        save_stops_to_json(stops, output_file)
    else:
        print(f"Filename {os.path.basename(file_path)} does not match the expected pattern.")

# extracts the route ID from the filename
def get_route_id_from_filename(filename):
    import re
    match = re.match(r'AMSY_([A-Za-z0-9]+)_', os.path.basename(filename))
    if match:
        return match.group(1)
    return None

# saves stop data as a JSON file
def save_stops_to_json(stops, output_file):
    with open(output_file, 'w') as f:
        json.dump(stops, f, indent=4)

# processes all XML files in a folder
def process_all_files_in_folder(folder_path, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            process_single_file(file_path, output_directory)

# set folder paths
folder_path = 'new_data/new_routes'
output_directory = 'new_data/stops'

process_all_files_in_folder(folder_path, output_directory)

## below saves a list of all the stops visited on a route, no extra details

In [16]:
import re


# parses the XML file to get stop point references
def parse_transxchange(file_path):
    namespace = {'txc': 'http://www.transxchange.org.uk/'}
    tree = ET.parse(file_path)
    root = tree.getroot()

    stop_refs = []
    # loops through stop points and collects references
    for stop_point in root.findall('.//txc:StopPoints/txc:AnnotatedStopPointRef', namespaces=namespace):
        ref = stop_point.find('txc:StopPointRef', namespaces=namespace).text
        stop_refs.append(ref)

    return stop_refs

# processes files in a folder and saves stop refs to JSON
def process_folder(folder_path, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    # loops through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            match = re.match(r'AMSY_([A-Za-z0-9]+)_', filename)
            if match:
                route_id = match.group(1)
                stop_refs = parse_transxchange(file_path)

                # saves stop refs to JSON file
                output_path = os.path.join(output_directory, f'AMSY_{route_id}_stop_refs.json')
                with open(output_path, 'w') as f:
                    json.dump(stop_refs, f, indent=4)
            else:
                print(f"{filename} does not match the expected pattern")

# specify folder paths and process the files
folder_path = 'new_data/new_routes'
output_directory = 'new_data/stop_refs'
process_folder(folder_path, output_directory)

saving all stops in one file, the one below may not work, use the one below that one

In [13]:

# parses the XML file to extract stop point references
def parse_transxchange(file_path):
    namespace = {'txc': 'http://www.transxchange.org.uk/'}
    tree = ET.parse(file_path)
    root = tree.getroot()

    stop_refs = []
    # gathers stop point references from the XML file
    for stop_point in root.findall('.//txc:StopPoints/txc:AnnotatedStopPointRef', namespaces=namespace):
        ref = stop_point.find('txc:StopPointRef', namespaces=namespace).text
        stop_refs.append(ref)

    return stop_refs

# processes all XML files in the folder and saves stop refs to JSON files
def process_folder(folder_path, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    # loops through files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            match = re.match(r'AMSY_([A-Za-z0-9]+)_', filename)
            if match:
                route_id = match.group(1)
                stop_refs = parse_transxchange(file_path)

                # saves stop refs to JSON file
                output_path = os.path.join(output_directory, f'AMSY_{route_id}_stop_refs.json')
                with open(output_path, 'w') as f:
                    json.dump(stop_refs, f, indent=4)
            else:
                print(f"{filename} does not match the expected pattern")

# specify folder paths and process the files
folder_path = 'new_data/new_routes'
output_directory = 'new_data/stop_refs'
process_folder(folder_path, output_directory)

## returns new stops file, a json with information on all stops

In [17]:


# parses the XML file to extract stop points
def parse_transxchange(file_path):
    namespace = {'txc': 'http://www.transxchange.org.uk/'}
    tree = ET.parse(file_path)
    root = tree.getroot()

    stops = {}
    # loops through stop points and collects information
    for stop_point in root.findall('.//txc:StopPoints/txc:AnnotatedStopPointRef', namespaces=namespace):
        ref = stop_point.find('txc:StopPointRef', namespaces=namespace).text
        common_name = stop_point.find('txc:CommonName', namespaces=namespace).text
        lon = float(stop_point.find('.//txc:Location/txc:Longitude', namespaces=namespace).text)
        lat = float(stop_point.find('.//txc:Location/txc:Latitude', namespaces=namespace).text)

        stops[ref] = {'common_name': common_name, 'coordinates': {'longitude': lon, 'latitude': lat}}

    return stops

# processes all XML files in a folder and writes the collected stops to one JSON file
def process_folder(folder_path, output_file):
    all_stops = {}

    # loops through the folder and processes XML files
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            file_path = os.path.join(folder_path, filename)
            stops = parse_transxchange(file_path)
            all_stops.update(stops)  # adds stops to the main dictionary

    # writes all stops to a single JSON file
    with open(output_file, 'w') as f:
        json.dump(all_stops, f, indent=4)

# specify folder and output paths
folder_path = 'new_data/new_routes'
output_file = 'new_data/all_stops.json'

# process the folder and save stops
process_folder(folder_path, output_file)

## above grabs stop data, below is stops and routes

In [18]:

import requests
import googlemaps
import folium
import polyline
from datetime import datetime
def parse_transxchange(file_path):
    namespace = {'txc': 'http://www.transxchange.org.uk/'}  #namespace dictionary

    tree = ET.parse(file_path)
    root = tree.getroot()

    # extract stop points
    stops = {}
    for stop_point in root.findall('.//txc:StopPoints/txc:AnnotatedStopPointRef', namespaces=namespace):
        ref = stop_point.find('txc:StopPointRef', namespaces=namespace).text
        common_name = stop_point.find('txc:CommonName', namespaces=namespace).text
        lon = float(stop_point.find('.//txc:Location/txc:Longitude', namespaces=namespace).text)
        lat = float(stop_point.find('.//txc:Location/txc:Latitude', namespaces=namespace).text)
        stops[ref] = {'common_name': common_name, 'coordinates': (lon, lat)}

    # extract route links
    route_links = []
    for route_section in root.findall('.//txc:RouteSections/txc:RouteSection', namespaces=namespace):
        for route_link in route_section.findall('txc:RouteLink', namespaces=namespace):
            from_stop = route_link.find('.//txc:From/txc:StopPointRef', namespaces=namespace).text
            to_stop = route_link.find('.//txc:To/txc:StopPointRef', namespaces=namespace).text
            route_links.append((from_stop, to_stop))

    return stops, route_links

In [19]:
def get_route_google(start_point, end_point, api_key):
    # initialise the Google Maps client with the API key
    gmaps = googlemaps.Client(key=api_key)
    now = datetime.now()

    # request directions for a bus route between the two points
    directions_result = gmaps.directions(start_point,
                                         end_point,
                                         mode="transit",
                                         transit_mode="bus",
                                         departure_time=now)

    # get the encoded polyline from the response and decode it
    polyline_encoded = directions_result[0]['overview_polyline']['points']
    return polyline.decode(polyline_encoded)

In [20]:
def plot_route_google(stops, route_links, api_key, cached_routes, route_id):
    if not stops:
        print("No stops data available.")
        return None, cached_routes, []

    # get coordinates of the first stop and initialise the map
    first_stop_coords = stops[next(iter(stops))]['coordinates']
    m = folium.Map(location=[first_stop_coords[1], first_stop_coords[0]], zoom_start=14)
    
    all_polylines = []
    processed_links = set()

    # process each link between stops
    for from_stop, to_stop in route_links:
        if from_stop in stops and to_stop in stops and (from_stop, to_stop) not in processed_links:
            processed_links.add((from_stop, to_stop))

            # check if the route is already cached
            if (from_stop, to_stop) in cached_routes:
                route_polyline = cached_routes[(from_stop, to_stop)]
            else:
                start_point = (stops[from_stop]['coordinates'][1], stops[from_stop]['coordinates'][0])
                end_point = (stops[to_stop]['coordinates'][1], stops[to_stop]['coordinates'][0])
                route_polyline = get_route_google(start_point, end_point, api_key)
                
                # cache the route if available
                if route_polyline:
                    cached_routes[(from_stop, to_stop)] = route_polyline

            if route_polyline:
                # draw the route on the map
                folium.PolyLine(route_polyline, color='blue').add_to(m)

                # add markers for the stops
                folium.Marker((stops[from_stop]['coordinates'][1], stops[from_stop]['coordinates'][0]),
                              popup=stops[from_stop]['common_name'], icon=folium.Icon(color='green')).add_to(m)
                folium.Marker((stops[to_stop]['coordinates'][1], stops[to_stop]['coordinates'][0]),
                              popup=stops[to_stop]['common_name'], icon=folium.Icon(color='red')).add_to(m)

                all_polylines.append({'from': from_stop, 'to': to_stop, 'polyline': route_polyline})
            else:
                print(f"route polyline not available for stops {from_stop} to {to_stop}.")
        else:
            print(f"{from_stop}, {to_stop} are missing in the stops dictionary or already processed.")

    return m, cached_routes, all_polylines

In [21]:


# save all route polylines to a JSON file
def save_polylines(all_route_polylines, filename="new_data/unencoded_polylines/all_route_polylines.json"):
    with open(filename, 'w') as f:
        json.dump(all_route_polylines, f, indent=4)
    print(f"All route polylines have been saved to {filename}")

# save polylines for a specific route to a separate JSON file
def save_route_polylines(route_id, polylines, directory="new_data/unencoded_polylines"):
    filename = os.path.join(directory, f"{route_id}_polylines.json")
    with open(filename, 'w') as f:
        json.dump(polylines, f, indent=4)
    print(f"Polylines for {route_id} have been saved to {filename}")

In [32]:
api_key = 'avilable on request , but this function costs routghly 150 quid per run '
def main(directory, api_key):
    all_route_polylines = {}
    cached_routes = {}

    # ensure the directory for saving polylines exists
    if not os.path.exists("new_data/unencoded_polylines"):
        os.makedirs("new_data/unencoded_polylines")

    # iterate over all XML files in the given directory
    for file_name in os.listdir(directory):
        print(file_name)
        if file_name.endswith('.xml'):
            file_path = os.path.join(directory, file_name)
            route_id = '_'.join(file_name.split('_')[:2])
            stops, route_links = parse_transxchange(file_path)
            map_google, cached_routes, polylines = plot_route_google(stops, route_links, api_key, cached_routes, route_id)
            all_route_polylines[route_id] = polylines
            save_route_polylines(route_id, polylines)  # save individual route polylines

    # save the final all route polylines
    save_polylines(all_route_polylines)


xml_directory = 'new_data/new_routes'


# Run the main script
main(xml_directory, api_key)

AMSY_10A_AMSYPC00011414710A_20240721_-_1894508.xml


KeyboardInterrupt: 

### it is fine at this point (above)

extracting polyline data from unencoded json

In [23]:
# load route polyline data from a JSON file
def load_route_data_from_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# extract polylines from route data and save them to a JSON file
def extract_and_save_polylines(route_data, output_filename):
    polylines = [segment['polyline'] for segment in route_data]  # list of polylines
    with open(output_filename, 'w') as file:
        json.dump(polylines, file, indent=4)
    print(f"Polylines have been saved to {output_filename}")

# main function to process files in the input directory and save extracted polylines
def main(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # process each JSON file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.json'):
            input_filepath = os.path.join(input_directory, filename)
            route_data = load_route_data_from_json(input_filepath)
            
            output_filename = f"{os.path.splitext(filename)[0]}_extracted_polylines.json"
            output_filepath = os.path.join(output_directory, output_filename)
            
            extract_and_save_polylines(route_data, output_filepath)

# input and output directories
input_directory = 'new_data/unencoded_polylines'
output_directory = 'new_data/extracted_polylines'

# run the main function
main(input_directory, output_directory)

Polylines have been saved to new_data/extracted_polylines\AMSY_10A_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_10G_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_10P_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_10W_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_10X_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_10_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_12C_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_12E_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_12G_polylines_extracted_polylines.json
Polylines have been saved to new_data/extracted_polylines\AMSY_12L_polylines_extrac

## attempting plotting of the extracted polylines


In [25]:
# load polyline data from a JSON file
def load_polylines_from_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# plot polylines on a map and save it as an HTML file
def plot_polylines(polylines):
    if not polylines:
        print("No polyline data available.")
        return

    # center the map using the first polyline's first coordinate
    first_coords = polylines[0][0]
    map_obj = folium.Map(location=first_coords, zoom_start=12)

    # add each polyline to the map
    for polyline in polylines:
        folium.PolyLine(polyline, color="blue", weight=5, opacity=0.8).add_to(map_obj)

    # save the map as an HTML file
    map_obj.save('new_data/polyline_map.html')
    print("Map has been saved as 'polyline_map.html'")

# main function to load polyline data and plot it
def main():
    filename = 'extracted_polylines.json'  # filename containing polyline data
    polyline_data = load_polylines_from_json(filename)
    plot_polylines(polyline_data)

# run the main function
main()

Map has been saved as 'polyline_map.html'


### at the above stage, the map is not erroneous

In [27]:
# read a JSON file and return the data
def read_json_file(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# convert polyline coordinates from lists of lists to lists of tuples
def convert_polylines_to_tuples(polylines):
    polytuples = [[tuple(coord) for coord in polyline] for polyline in polylines]
    return polytuples

# save the converted polyline data to a JSON file
def save_converted_polylines_to_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"Converted polyline data has been saved to {filename}")

# main function to process and convert polyline files
def main(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # process each JSON file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.json'):
            input_filepath = os.path.join(input_directory, filename)
            polylines_data = read_json_file(input_filepath)
            
            # convert coordinates from lists to tuples
            converted_polylines = convert_polylines_to_tuples(polylines_data)

            # save the converted polyline data
            output_filename = os.path.splitext(filename)[0] + '_converted.json'
            output_filepath = os.path.join(output_directory, output_filename)
            
            save_converted_polylines_to_json(converted_polylines, output_filepath)

#input and output directories
input_directory = 'new_data/extracted_polylines'
output_directory = 'new_data/converted_polylines'


main(input_directory, output_directory)

Converted polyline data has been saved to new_data/converted_polylines\AMSY_10A_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_10G_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_10P_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_10W_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_10X_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_10_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_12C_polylines_extracted_polylines_converted.json
Converted polyline data has been saved to new_data/converted_polylines\AMSY_12E_polylines_extracted_polylines_converted.json
C

attempting to compare if the json is equal to the tuples


In [28]:

def load_data_from_json(filename):
     """Load data from a JSON file."""
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

# convert polyline coordinates from lists of lists to lists of tuples
def convert_polylines_to_tuples(polylines):
    return [[tuple(coord) for coord in polyline] for polyline in polylines]

# compare two datasets and print differences
def compare_data(original, modified):
    for i, (orig_polyline, mod_polyline) in enumerate(zip(original, modified)):
        for j, (orig_coord, mod_coord) in enumerate(zip(orig_polyline, mod_polyline)):
            if orig_coord != list(mod_coord):
                print(f"Difference found in polyline {i}, coordinate {j}:")
                print(f"Original: {orig_coord}")
                print(f"Modified: {mod_coord}")

# plot multiple polylines on a map and save to an HTML file
def plot_polylines(polylines, filename='polyline_map.html'):
    if not polylines:
        print("No polyline data available.")
        return

    # center the map using the first polyline's first coordinate
    first_coords = polylines[0][0]
    map_obj = folium.Map(location=first_coords, zoom_start=12)

    # add each polyline to the map
    for polyline in polylines:
        folium.PolyLine(polyline, color="blue", weight=5, opacity=0.8).add_to(map_obj)

    # save the map as an HTML file
    map_obj.save(filename)
    print(f"Map has been saved as '{filename}'")

original_filename = 'new_data/extracted_polylines.json'
original_data = load_data_from_json(original_filename)

# Convert coordinates from lists to tuples
converted_polylines = convert_polylines_to_tuples(original_data)

# compare the original and modified data
compare_data(original_data, converted_polylines)

# plot the original and converted polylines on separate maps
plot_polylines(original_data, 'original_polyline_map.html')
plot_polylines(converted_polylines, 'converted_polyline_map.html')

FileNotFoundError: [Errno 2] No such file or directory: 'new_data/extracted_polylines.json'

### The above seems to return to correct maps

In [30]:
# encode each polyline using the polyline library
def encode_polylines(polylines):
    return [polyline.encode(pl) for pl in polylines]

# save the encoded polyline data to a JSON file
def save_encoded_polylines_to_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"Encoded polyline data has been saved to {filename}")

# encode all converted polylines in a directory and save them to another directory
def encode_all_converted_polylines(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # process each JSON file with converted polylines
    for filename in os.listdir(input_directory):
        if filename.endswith('_converted.json'):
            input_filepath = os.path.join(input_directory, filename)
            converted_polylines_data = read_json_file(input_filepath)
            
            # encode the polylines
            encoded_polylines = encode_polylines(converted_polylines_data)
            
            # save encoded polylines
            encoded_output_filename = os.path.splitext(filename)[0] + '_encoded.json'
            encoded_output_filepath = os.path.join(output_directory, encoded_output_filename)
            save_encoded_polylines_to_json(encoded_polylines, encoded_output_filepath)

# directories for converted input and encoded output
converted_input_directory = 'new_data/converted_polylines'
encoded_output_directory = 'new_data/encoded_polylines'

# run the encoding function
encode_all_converted_polylines(converted_input_directory, encoded_output_directory)

Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_10A_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_10G_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_10P_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_10W_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_10X_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_10_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_12C_polylines_extracted_polylines_converted_encoded.json
Encoded polyline data has been saved to new_data/encoded_polylines\AMSY_12E_polylines_extracted_po

In [31]:
# decode each encoded polyline using the polyline library
def decode_polylines(encoded_polylines):
    return [polyline.decode(pl) for pl in encoded_polylines]

# plot decoded polylines on a map and save it as an HTML file
def plot_polylines(decoded_polylines, output_filename):
    if not decoded_polylines:
        print("No decoded polyline data available.")
        return

    # center the map using the first polyline's first coordinate
    first_coords = decoded_polylines[0][0]
    map_obj = folium.Map(location=first_coords, zoom_start=13)

    # add each decoded polyline to the map
    for polyline in decoded_polylines:
        folium.PolyLine(polyline, color="blue", weight=5, opacity=0.8).add_to(map_obj)

    # save the map as an HTML file
    map_obj.save(output_filename)
    print(f"Map has been saved as '{output_filename}'")

# decode encoded polylines and plot them on a map
def decode_and_plot_encoded_polylines(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # process each encoded JSON file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('_encoded.json'):
            input_filepath = os.path.join(input_directory, filename)
            encoded_polylines_data = read_json_file(input_filepath)
            
            # decode the polylines
            decoded_polylines = decode_polylines(encoded_polylines_data)
            
            # plot the decoded polylines and save the map
            map_output_filename = os.path.splitext(filename)[0] + '_map.html'
            map_output_filepath = os.path.join(output_directory, map_output_filename)
            plot_polylines(decoded_polylines, map_output_filepath)

# directories for encoded polylines and map output
encoded_input_directory = 'new_data/encoded_polylines'
plot_output_directory = 'new_data/decoded_maps'

decode_and_plot_encoded_polylines(encoded_input_directory, plot_output_directory)

Map has been saved as 'new_data/decoded_maps\AMSY_10A_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_10G_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_10P_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_10W_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_10X_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_10_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_12C_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_12E_polylines_extracted_polylines_converted_encoded_map.html'
Map has been saved as 'new_data/decoded_maps\AMSY_12G_polylines_extracted_polylines_converted_encoded_map

### encoding and decoding are now correct. problem may of been in mapping and parsing of data